# Avito Category Prediction
 You need to predict the category of avito publications

In [1]:
import pandas as pd
import pickle

Предобработанные данные:

In [2]:
with open('traintokenized', 'rb') as traintokenized:
    train = pickle.load(traintokenized)
    
with open('testtokenized', 'rb') as testtokenized:
    test = pickle.load(testtokenized)

In [3]:
train.head()

,Category_name,Category,desctitletokens
0,Запчасти и аксессуары,10,эбу renault nissan dci delphi год комплект эбу...
1,Бытовая техника,21,утюг утп ватт ссср год продавать фото утюг утп...
2,Предложение услуг,114,возвма машина выкуп взять машина аренда послед...
3,"Одежда, обувь, аксессуары",27,полусапожки полусапожки отличный состояние оде...
4,"Одежда, обувь, аксессуары",27,босоножка кожаный кожаный натура босоножка kar...


In [4]:
test.head()

,itemid,desctitletokens
0,1778449823,мастерок мастерок фирма форвард белый цвет иде...
1,1677656962,зимний сапог продавать зимний сапог хороший со...
2,1758182804,видеонаблюдение камера комплект atis amd входи...
3,1689811299,запчасть gle запчасть gle
4,1804706240,бластер nerf состояние продавать нужный диск п...


#### Данные готовы, и можно использовать обучение без токенизации.

In [5]:
random_data = train.sample(500000)
del train

In [6]:
random_data.head()

,Category_name,Category,desctitletokens
336181,Книги и журналы,83,универсальный справочник обществознание егэ ку...
4059360,Товары для детей и игрушки,30,накладка сосок грудь avent новый распеч новый ...
4161435,Охота и рыбалка,102,разборный мангал любой рисунок мангал разборны...
2569637,Детская одежда и обувь,29,теплый лосина штаны год отдавать пакет теплый ...
498920,Коллекционирование,36,thomas anders pures leben thomas anders pures ...


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(random_data['desctitletokens'], random_data['Category'], test_size=0.3, random_state=2021)

In [8]:
def tokenizer(s: str) -> list:
    return s.split()

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report,accuracy_score

sgd = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenizer)),
    ('clf', SGDClassifier(random_state=2021, n_jobs=8)),
    ])

sgd.fit(X_train, y_train)

y_train_pred = sgd.predict(X_train)


# classification_report(y_test, sgd_y_pred)
accuracy_score(y_train, y_train_pred)

In [ ]:
accuracy_score(train['Category'], sgd.predict(train['text']))


In [ ]:
output = pd.DataFrame({
    'Id': test.itemid, 
    'Category': sgd.predict(test['text'])
})
output.to_csv("submission.csv", index=False)
output.head()

In [9]:
%%time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,accuracy_score


pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenizer)),
    ('clf', LinearSVC())
    ])

pipe.fit(X_train, y_train)
pipe_y_pred = pipe.predict(X_test)


# classification_report(y_test, sgd_y_pred)
accuracy_score(y_test, pipe_y_pred)

/home/ksusonic/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


CPU times: user 7min 59s, sys: 765 ms, total: 8min
Wall time: 8min 1s


0.8646133333333333

In [ ]:
output = pd.DataFrame({
    'Id': test.itemid, 
    'Category': pipe.predict(test['text'])
})
output.to_csv("submission.csv", index=False)
output.head()